In [31]:
import numpy as np
import pandas as pd
import seaborn as sb

from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler, MinMaxScaler
from sklearn.impute import SimpleImputer

In [486]:
# load data

train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/HousePrices/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/HousePrices/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/HousePrices/sample_submission.csv')

train.set_index('Id', inplace=True)
test.set_index('Id', inplace=True)

In [487]:
# combine train and test data

all_data = pd.concat([train, test], sort=False)
all_data_index = all_data.index

train_y = train['SalePrice']

In [488]:
# find deficient data

na_check = all_data.isna().sum() / len(all_data)
na_check = na_check[na_check > 0.3]
na_check

Alley          0.932169
FireplaceQu    0.486468
PoolQC         0.996574
Fence          0.804385
MiscFeature    0.964029
SalePrice      0.499829
dtype: float64

In [489]:
# analyze NA data

# fill NA of fireplaceQu because these don't have fireplaces
all_data.loc[all_data['FireplaceQu'].isna(), 'Fireplaces'] = 0

# NA of Alley means there is no path
all_data.loc[all_data['Alley'].isna(), 'Alley'] = 'noPath'

# NA of PoolQC means there is no pool
all_data.loc[all_data['PoolQC'].isna(), 'PoolQC'] = 'noPool'

# NA of Fence means there is no fence
all_data.loc[all_data['Fence'].isna(), 'Fence'] = 'noFence'

# NA of MiscFeature means there is no other feature
all_data.loc[all_data['MiscFeature'].isna(), 'MiscFeature'] = 'noFeature'

In [228]:
#all_data = all_data.drop(na_check.keys().drop(['SalePrice']), axis=1)

In [490]:
rating_col = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']

########################################################################

col = 'ExterQual'
ratings = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
score = len(ratings)
all_data[col+'(status)'] = 0
for i in ratings:
  all_data.loc[all_data[col] == i, col+'(status)'] = score
  score -= 1

########################################################################

col = 'ExterCond'
ratings = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
score = len(ratings)
all_data[col+'(status)'] = 0
for i in ratings:
  all_data.loc[all_data[col] == i, col+'(status)'] = score
  score -= 1

########################################################################

col = 'BsmtQual'
ratings = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
score = len(ratings)
all_data[col+'(status)'] = 0
for i in ratings:
  all_data.loc[all_data[col] == i, col+'(status)'] = score
  score -= 1

########################################################################

col = 'BsmtCond'
ratings = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
score = len(ratings)
all_data[col+'(status)'] = 0
for i in ratings:
  all_data.loc[all_data[col] == i, col+'(status)'] = score
  score -= 1

########################################################################

col = 'HeatingQC'
ratings = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
score = len(ratings)
all_data[col+'(status)'] = 0
for i in ratings:
  all_data.loc[all_data[col] == i, col+'(status)'] = score
  score -= 1

########################################################################

col = 'KitchenQual'
ratings = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
score = len(ratings)
all_data[col+'(status)'] = 0
for i in ratings:
  all_data.loc[all_data[col] == i, col+'(status)'] = score
  score -= 1

########################################################################

col = 'FireplaceQu'
ratings = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
score = len(ratings)
all_data[col+'(status)'] = 0
for i in ratings:
  all_data.loc[all_data[col] == i, col+'(status)'] = score
  score -= 1

########################################################################

col = 'GarageQual'
ratings = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
score = len(ratings)
all_data[col+'(status)'] = 0
for i in ratings:
  all_data.loc[all_data[col] == i, col+'(status)'] = score
  score -= 1

########################################################################

col = 'GarageCond'
ratings = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
score = len(ratings)
all_data[col+'(status)'] = 0
for i in ratings:
  all_data.loc[all_data[col] == i, col+'(status)'] = score
  score -= 1

########################################################################

col = 'PoolQC'
ratings = ['Ex', 'Gd', 'TA', 'Fa']
score = len(ratings)
all_data[col+'(status)'] = 0
for i in ratings:
  all_data.loc[all_data[col] == i, col+'(status)'] = score
  score -= 1

########################################################################

all_data = all_data.drop(rating_col, axis=1)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,ExterQual(status),ExterCond(status),BsmtQual(status),BsmtCond(status),HeatingQC(status),KitchenQual(status),FireplaceQu(status),GarageQual(status),GarageCond(status),PoolQC(status)
count,2919.000000,2433.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2896.000000,2918.000000,2918.000000,2918.000000,2918.000000,2919.000000,2919.000000,2919.000000,2919.000000,2917.000000,2917.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2760.000000,2918.000000,2918.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,1460.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000
mean,57.137718,69.305795,10168.114080,6.089072,5.564577,1971.312778,1984.264474,102.201312,441.423235,49.582248,560.772104,1051.777587,1159.581706,336.483727,4.694416,1500.759849,0.429894,0.061364,1.568003,0.380267,2.860226,1.044536,6.451524,0.597122,1978.113406,1.766621,472.874572,93.709832,47.486811,23.098321,2.602261,16.062350,2.251799,50.825968,6.213087,2007.792737,180921.195890,3.396711,3.085646,3.477561,2.918465,4.151764,3.509764,1.768071,2.800959,2.808839,0.010277
std,42.517628,23.344905,7886.996359,1.409947,1.113131,30.291442,20.894344,179.334253,455.610826,169.205611,439.543659,440.766258,392.362079,428.701456,46.396825,506.051045,0.524736,0.245687,0.552969,0.502872,0.822693,0.214462,1.569379,0.646129,25.574285,0.761624,215.394815,126.526589,67.575493,64.244246,25.188169,56.184365,35.663946,567.402211,2.714762,1.314964,79442.502883,0.580293,0.372361,0.905448,0.574950,0.957952,0.665273,1.806619,0.715863,0.712498,0.186681
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1895.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000,2.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20.000000,59.000000,7478.000000,5.000000,5.000000,1953.500000,1965.000000,0.000000,0.000000,0.000000,220.000000,793.000000,876.000000,0.000000,0.000000,1126.000000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1960.000000,1.000000,320.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,129975.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,0.000000,3.000000,3.000000,0.000000
50%,50.000000,68.000000,9453.000000,6.000000,5.000000,1973.000000,1993.000000,0.000000,368.500000,0.000000,467.000000,989.500000,1082.000000,0.000000,0.000000,1444.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1979.000000,2.000000,480.000000,0.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000,3.000000,3.000000,4.000000,3.000000,5.000000,3.000000,1.000000,3.000000,3.000000,0.000000
75%,70.000000,80.000000,11570.000000,7.000000,6.000000,2001.000000,2004.000000,164.000000,733.000000,0.000000,805.500000,1302.000000,1387.500000,704.000000,0.000000,1743.500000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000,4.000000,3.000000,4.000000,3.000000,5.000000,4.000000,4.000000,3.000000,3.000000,0.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000

In [491]:
# separate object types and non-object types

data_object = all_data.select_dtypes(include='object')
data_nonObject = all_data.select_dtypes(exclude='object')

print(data_object.columns)
print(data_nonObject.columns)

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'Foundation', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'Heating', 'CentralAir', 'Electrical',
       'Functional', 'GarageType', 'GarageFinish', 'PavedDrive', 'Fence',
       'MiscFeature', 'SaleType', 'SaleCondition'],
      dtype='object')
Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3

In [492]:
# one-hot coding

dummy_data = pd.get_dummies(data_object, drop_first=True)
dummy_data.index = all_data_index

In [493]:
impute = SimpleImputer(strategy='most_frequent')

impute.fit(data_nonObject)
data_nonObject_ = impute.transform(data_nonObject)

data_nonObject = pd.DataFrame(data_nonObject_, columns=data_nonObject.columns, index=all_data_index)

In [494]:
scaler = MinMaxScaler()
dump = data_nonObject

scaler.fit(dump)
dump_ = scaler.transform(dump)
dump = pd.DataFrame(dump_, columns=dump.columns, index=data_nonObject.index)
dump['SalePrice'] = data_nonObject['SalePrice']

In [495]:
all_data = pd.merge(dummy_data, dump, right_index=True, left_index=True)

In [505]:
# select reliable data
nTrain = len(train)
train = all_data[:nTrain]
test = all_data[nTrain:]

cormat = train.corr()
df = cormat.index[abs(cormat['SalePrice']) >= 0.1]
train = train[df]
test = test[df.drop('SalePrice')]
all_data = all_data[df]
df

Index(['MSZoning_RL', 'MSZoning_RM', 'Alley_noPath', 'LotShape_IR2',
       'LotShape_Reg', 'LandContour_HLS', 'LotConfig_CulDSac',
       'Neighborhood_BrDale', 'Neighborhood_BrkSide', 'Neighborhood_Edwards',
       ...
       'SalePrice', 'ExterQual(status)', 'BsmtQual(status)',
       'BsmtCond(status)', 'HeatingQC(status)', 'KitchenQual(status)',
       'FireplaceQu(status)', 'GarageQual(status)', 'GarageCond(status)',
       'PoolQC(status)'],
      dtype='object', length=101)

In [501]:
from sklearn.model_selection import train_test_split

train_x = train.drop(['SalePrice'], axis=1)
train_y = np.log1p(train['SalePrice'])

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, shuffle=True)

test_id_idx = test.index

test_x = test

In [502]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

xgb = XGBRegressor(randome_state = 37)

param = {
    'max_depth':[2,3,4],
    'n_estimators':range(550,700,50),
    'colsample_bytree':[0.5,0.7,1],
    'colsample_bylevel':[0.5,0.7,1],
}
grid_search = GridSearchCV(estimator=xgb, param_grid=param, cv=5, 
                           scoring='neg_mean_squared_error',
                           n_jobs=-1)

#print(train_x.dtypes)
#print(train_y.dtypes)

grid_search.fit(train_x, train_y)
print(grid_search.best_params_)
print(grid_search.best_estimator_)

[09:39:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
{'colsample_bylevel': 0.5, 'colsample_bytree': 0.7, 'max_depth': 3, 'n_estimators': 650}
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.5,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=650,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             randome_state=37, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)


In [503]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
pred_train = grid_search.predict(train_x)
pred_val = grid_search.predict(val_x)

print('train mae score: ', mean_absolute_error(train_y, pred_train))
print('val mae score:', mean_absolute_error(val_y, pred_val))

#train mae score:  0.014433974714290313
#val mae score: 0.09069824674400735

#train mae score:  0.010851636153666308
#val mae score: 0.01700044946603384

train mae score:  0.026426322418250617
val mae score: 0.09356628187785215


In [504]:
pred = grid_search.predict(test_x)
submission['SalePrice'] = np.expm1(pred)
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/HousePrices/submission.csv', index=False)

In [435]:
submission.head(20)

,Id,SalePrice
0,1461,0.111913
1,1462,0.180602
2,1463,0.209513
3,1464,0.219599
4,1465,0.206173
5,1466,0.200196
6,1467,0.185764
7,1468,0.184841
8,1469,0.198273
9,1470,0.130308
